In [1]:
import scipy
import numpy as np
import scipy.sparse as sp
import scipy.linalg as sl
from qiskit.quantum_info import Statevector

from qiskit_nature.units import DistanceUnit
# from qiskit_nature.second_q.drivers import PySCFDriver
# from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.mappers import InterleavedQubitMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.circuit import Parameter

from scipy.optimize import minimize

In [2]:
def d2b(n, l):
    # '5' -> '0101' when n = 5, l = 4
    fs = "{:0" + str(int(l)) + "b}"
    return fs.format(n)

def qs2fs(qs):
    return qs

def qv2fv(qvec):
    ## qiskit vector to openfermion vector due to specific orbital ordering
    cand = 0*qvec
    vlen = len(qvec)
    nq = int(np.log2(vlen))
    for i in range(vlen):
        qbin_i = d2b(i, nq)[::-1]
        fbin_i = qs2fs(qbin_i)
        # print(qbin_i, fbin_i)
        cand[int(fbin_i, 2)] = qvec[i]
    return cand

def HFstate(nele, nq):
    hf_str = '1'*nele + '0'*(nq - nele)
    hf = np.zeros(2**(nq))
    hf[int(hf_str,2)] = 1
    return hf

## Extract the basis vectors $\{\ket{\phi_j}\}$ and the eigenvector corresponds to the lowest energy $\vec{v}$ from GCM

In [ ]:
# mole_name = 'H4S'
# iter_num = 14 #10
# geom = "H 0 0 0; H 0.03141463462364135 1.9997532649633212 0; H 2.0314146346236415 1.9997532649633212 0; H 2.0628292692472825 0 0"
# nele = 4
# purt = 1e-14
# unit = DistanceUnit.BOHR

# mole_name = 'H4L'
# iter_num = 10
# geom = "H 0 0 0; H 2 0 0; H 4 0 0; H 6 0 0"
# nele = 4
# purt = 1e-14
# unit = DistanceUnit.BOHR

# mole_name = 'LiH'
# iter_num = 55
# geom = "Li 0 0 0; H 0 0 1.62"
# nele = 4
# purt = 1e-12
# unit = DistanceUnit.ANGSTROM

# mole_name = 'BEH2'
# iter_num = 103
# geom = "Be 0 0 0; H -1.33 0 0; H 1.33 0 0"
# nele = 6
# purt = 1e-14
# unit = DistanceUnit.ANGSTROM

# mole_name='H65A'
# geom = "H 0 0 0; H 5 0 0; H 10 0 0; H 15 0 0; H 20 0 0; H 25 0 0"
# iter_num = 36
# nele = 6
# purt = 1e-14
# unit = DistanceUnit.ANGSTROM

# mole_name='H620'
# geom = "H 0 0 0; H 2 0 0; H 4 0 0; H 6 0 0; H 8 0 0; H 10 0 0"
# iter_num = 78
# nele = 6
# purt = 1e-14
# unit = DistanceUnit.BOHR

mole_name='H635'
geom = "H 0 0 0; H 3.5 0 0; H 7 0 0; H 10.5 0 0; H 14 0 0; H 17.5 0 0"
# iter_num = 69
iter_num = 92
nele = 6
purt = 1e-14
unit = DistanceUnit.BOHR

# mole_name='N235F4'
# iter_num = 20
# nele = 6
# purt = 1e-12
# unit = DistanceUnit.ANGSTROM

# mole_name = 'H4E65'
# iter_num = 11
# geom = "H 0 0 0; H 0.65 0 0; H 1.3 0 0; H 1.95 0 0"
# nele = 4
# purt = 1e-14
# unit = DistanceUnit.ANGSTROM

# mole_name = 'H4E7'
# iter_num = 14
# geom = "H 0 0 0; H 0.7 0 0; H 1.4 0 0; H 2.1 0 0"
# nele = 4
# purt = 1e-14
# unit = DistanceUnit.ANGSTROM

# mole_name = 'H4E75'
# iter_num = 14
# geom = "H 0 0 0; H 0.75 0 0; H 1.5 0 0; H 2.25 0 0"
# nele = 4
# purt = 1e-14
# unit = DistanceUnit.ANGSTROM

# mole_name = 'H4E8'
# iter_num = 16
# geom = "H 0 0 0; H 0.8 0 0; H 1.6 0 0; H 2.4 0 0"
# nele = 4
# purt = 1e-14
# unit = DistanceUnit.ANGSTROM



gcm_folder = 'Output_Data/ADAPT-GCM/GCM_'
# vqe_folder = 'Output_Data/ADAPT-VQE-GCM/GV_'
fix = '_tp4/'
gcm_file_address = gcm_folder + mole_name + fix

mole_ham = sp.load_npz(gcm_file_address+f'MoleHam.npz')
nq = int( np.log2(mole_ham.shape[1]) )
hf_vec = HFstate(nele, nq)
basis_mat = np.matrix( np.load(gcm_file_address+f'IterMats/IterAG{iter_num}_basis_mat.npy') )
basis_mat_sp = sp.csc_matrix(basis_mat)

oldH = np.load(gcm_file_address+f'IterMats/IterAG{iter_num}_H.npy')
oldS = np.load(gcm_file_address+f'IterMats/IterAG{iter_num}_S.npy')
newH = basis_mat_sp.conjugate().transpose() @ mole_ham @ basis_mat_sp

# mole_eval, mole_evec = sl.eigh(mole_ham)
true_ev, low_evec = scipy.sparse.linalg.eigsh(mole_ham,1,which='SA',v0=hf_vec)
true_ev = true_ev[0]
print("True lowest eigenvalue:", true_ev)

## verification
print("Verification of the right basis matrix:", np.linalg.norm(oldH - newH))

## Eigenvalues and eigenvectors
eval, evec = sl.eigh(oldH, oldS+np.eye(oldS.shape[0])*purt)
print("Verification of eigenvector:", np.linalg.norm(oldH.dot(evec[:,0]) - eval[0] * oldS.dot(evec[:,0]) ))

print("\nLowest eigenvalue error:", np.abs(eval[0] - true_ev))
# print("Eigenvector norm:", np.linalg.norm(evec[:,0]))

basis_mat.shape, evec[:,0].shape

## Compute GCM state $\ket{\psi_{GCM}} = \sum_j v_j \ket{\phi_{j}}$ up to a normalization

In [ ]:
gcm_vec = basis_mat[:,0] * evec[:,0][0]
for ii in range(1, basis_mat.shape[1]):
    gcm_vec += basis_mat[:,ii] * evec[:,0][ii]

norm_gcm_vec = np.linalg.norm(gcm_vec, ord=2)

gcm_state = gcm_vec / norm_gcm_vec

if np.linalg.norm(gcm_state-low_evec) > 1:
    gcm_state = -gcm_state

gcm_sv = Statevector(gcm_state)

print(gcm_state[np.abs(gcm_state) > 0].real)
gcm_vec.shape, norm_gcm_vec, np.linalg.norm(gcm_state, ord=2)

## Check with exact eigenvectors

In [ ]:
overlap = np.abs(gcm_state.conjugate().T.dot(low_evec).flatten()[0,0])**2
print('Eigenvalue = {:8.5f}  '.format(true_ev), 
    "1-|<GCM|Exact>|^2=", 1-overlap, "Norm differnce", np.linalg.norm(gcm_state-low_evec)) 

## Prepare UCCSD state $\ket{\psi_{UCCSD}}$ from $\ket{\psi_{HF}}$

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

driver = PySCFDriver(
    atom=geom,
    basis="sto3g",
    charge=0,
    spin=0,
    unit=unit,
)
# DistanceUnit.ANGSTROM

es_problem = driver.run()
mapper = InterleavedQubitMapper(JordanWignerMapper())
# qiskit_ham = mapper.map(es_problem.hamiltonian.second_q_op()).to_matrix()

# Create the UCCSD circuit
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)
# ansatz = UCC(num_spatial_orbitals=4, num_particles=(2,2),excitations='sd', qubit_mapper=mapper,
#                initial_state=HartreeFock(4, (2,2), mapper) )

from qiskit_nature.second_q.algorithms.initial_points import MP2InitialPoint,VSCFInitialPoint, HFInitialPoint
initial_point = MP2InitialPoint() # VSCFInitialPoint() # MP2InitialPoint()
initial_point.ansatz = ansatz
initial_point.problem = es_problem
initial_point.to_numpy_array()

In [22]:
# Prepare the ansatz circuit
params = ansatz.parameters
param_values = [Parameter(f'theta_{i}') for i in range(len(params))]
ansatz_circuit = ansatz.assign_parameters(param_values)

## Define the cost function to maximize overlap
def obj_func(parameter_values):
    uccsd_circuit = ansatz_circuit.assign_parameters(parameter_values)
    psi_uccsd_unopt = qv2fv(Statevector(uccsd_circuit).data)
    overlap = np.array((psi_uccsd_unopt.conjugate().T.dot(gcm_state))**2).real[0,0]
    return -overlap 

# Optimize the parameters to maximize overlap
from numpy.random import Generator, PCG64
rng = Generator(PCG64(38749))
# initial_params = (2*rng.random(len(params))-1)*np.pi ## initial guess
initial_params = initial_point.to_numpy_array()

Nfeval = 1
def callbackF(Xi):
    global Nfeval
    print('{0:4d}   {1: 3.6f}   {2: 3.6f}   {3: 3.6f}   {4: 3.6f}'.format(Nfeval, Xi[0], Xi[1], Xi[2], obj_func(Xi)))
    Nfeval += 1


# result = minimize(obj_func, initial_params, method='CG', callback=callbackF, options={'disp': True, 'maxiter': 300})
result = minimize(obj_func, initial_params, method='Nelder-Mead', options={'disp': True, 'maxiter': 1000})
# initial_opt_params = result.x
print("1-overlap=", 1+result.fun)
result

1-overlap= 0.6754225442067525


/var/folders/74/7bbhmvxd7mb2xrnkplmh90xc0000gn/T/ipykernel_40222/808767598.py:27: RuntimeWarning: Maximum number of iterations has been exceeded.
  result = minimize(obj_func, initial_params, method='Nelder-Mead', options={'disp': True, 'maxiter': 1000})


       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: -0.3245774557932475
             x: [-1.037e-04 -3.178e-05 ...  1.313e-05 -7.117e-02]
           nit: 1000
          nfev: 1133
 final_simplex: (array([[-1.037e-04, -3.178e-05, ...,  1.313e-05,
                        -7.117e-02],
                       [-1.394e-04, -4.093e-06, ...,  3.815e-05,
                        -7.103e-02],
                       ...,
                       [-2.473e-05, -5.178e-05, ...,  1.661e-05,
                        -7.060e-02],
                       [-2.587e-05, -4.531e-05, ...,  1.905e-05,
                        -7.060e-02]]), array([-3.246e-01, -3.241e-01, ..., -3.225e-01, -3.225e-01]))

In [8]:
np.save(gcm_file_address+'UCCSD_overlap_optparam.npy', result.x)

## Verfication

In [ ]:
psi_uccsd = qv2fv(Statevector(ansatz_circuit.assign_parameters(result.x)).data)
psi_uccsd_vec = np.matrix(psi_uccsd.real.reshape(-1,1))
uccsd_ev_qh = (psi_uccsd_vec.conjugate().T.dot(mole_ham.todense()).dot(psi_uccsd_vec)[0,0].real)/psi_uccsd_vec.conjugate().T.dot(psi_uccsd_vec)[0,0]
uccsd_ev_qh, true_ev, uccsd_ev_qh - true_ev

In [10]:
# uccsd_ev_qiskit_ham = (psi_uccsd_vec.conjugate().T.dot(qiskit_ham).dot(psi_uccsd_vec)[0,0].real)/psi_uccsd_vec.conjugate().T.dot(psi_uccsd_vec)[0,0]
# uccsd_ev_qiskit_ham, true_ev, uccsd_ev_qiskit_ham - true_ev

In [ ]:
hf_ev = hf_vec.conjugate().T.dot(mole_ham.todense()).dot(hf_vec)[0,0]
gcm_ev = gcm_state.conjugate().T.dot(mole_ham.todense()).dot(gcm_state)[0,0]
gcm_ev, hf_ev , gcm_ev - true_ev